In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def read_pickle(filename: str) -> pd.DataFrame:
    with open(filename, 'rb') as f:
        return pd.DataFrame(pickle.load(f))

In [4]:
dataset_train_path = "data/02-processed/spanish/train_cluster.pkl_train.pkl"
dataset_test_path = "data/02-processed/spanish/train_cluster.pkl_test.pkl"

dataset_train = read_pickle(dataset_train_path)
dataset_test = read_pickle(dataset_test_path)

In [5]:
# tsne
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0, perplexity=min(30, len(dataset_train["sample"])-1), n_iter=1000)

X_2d = tsne.fit_transform(np.array(dataset_train["sample"].to_list()))

/home/miguel/.local/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [6]:
# represent the data in a scatter plot the are only 2 clusters the colors must be red for 0 and blue for 1
colors = ['red', 'blue']
plt.figure(figsize=(6, 5))
for i in range(len(X_2d)):
    plt.scatter(X_2d[i, 0], X_2d[i, 1], color=colors[dataset_train['label'][i]], s=10)
# add legend
plt.legend(['1', '0'], loc='upper right')
plt.show()

In [8]:
# Count the occurrences of each label
label_counts = dataset_train['label'].value_counts()

# Plot the histogram manually to assign colors per label
plt.bar(label_counts.index, label_counts.values, color=['blue', 'red'], alpha=0.7, edgecolor='black')

# Adding labels, legend, and title
plt.title("Distribution of Labels")
plt.xlabel("Labels")
plt.ylabel("Frequency")
plt.xticks([0, 1], ['Class 0', 'Class 1'])
#plt.legend(['Class 0', 'Class 1'], loc="upper right")

# Show the plot
plt.show()

## CLF

In [7]:
from lazypredict.Supervised import LazyClassifier

In [10]:
X_train = np.array(dataset_train["sample"].to_list())
# get subset of the data using shuffle
X_train = X_train[:1000]
y_train = np.array(dataset_train["label"].to_list())
y_train = y_train[:1000]
X_test = np.array(dataset_test["sample"].to_list())
y_test = np.array(dataset_test["label"].to_list())

In [11]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

 97%|█████████▋| 30/31 [00:30<00:00,  1.90it/s]

[LightGBM] [Info] Number of positive: 374, number of negative: 626
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006893 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195838
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.374000 -> initscore=-0.515095
[LightGBM] [Info] Start training from score -0.515095


100%|██████████| 31/31 [00:30<00:00,  1.00it/s]


In [12]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
NuSVC                              0.66               0.66     0.66      0.67   
SVC                                0.67               0.65     0.65      0.66   
GaussianNB                         0.61               0.63     0.63      0.61   
LGBMClassifier                     0.65               0.62     0.62      0.64   
XGBClassifier                      0.65               0.62     0.62      0.64   
KNeighborsClassifier               0.59               0.61     0.61      0.59   
ExtraTreesClassifier               0.66               0.61     0.61      0.64   
LogisticRegression                 0.61               0.60     0.60      0.61   
AdaBoostClassifier                 0.62               0.60     0.60      0.62   
RandomForestClassifier             0.65               0.60     0.60      0.63   
RidgeClassifierCV           

In [13]:
# normalize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

In [14]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

 97%|█████████▋| 30/31 [00:29<00:00,  1.90it/s]

[LightGBM] [Info] Number of positive: 374, number of negative: 626
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006761 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195838
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.374000 -> initscore=-0.515095
[LightGBM] [Info] Start training from score -0.515095


100%|██████████| 31/31 [00:29<00:00,  1.03it/s]


In [15]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
NuSVC                              0.66               0.66     0.66      0.67   
SVC                                0.67               0.65     0.65      0.66   
GaussianNB                         0.61               0.63     0.63      0.61   
LGBMClassifier                     0.65               0.62     0.62      0.64   
XGBClassifier                      0.65               0.62     0.62      0.64   
KNeighborsClassifier               0.59               0.61     0.61      0.59   
ExtraTreesClassifier               0.66               0.61     0.61      0.64   
LogisticRegression                 0.61               0.60     0.60      0.61   
AdaBoostClassifier                 0.62               0.60     0.60      0.62   
RandomForestClassifier             0.65               0.60     0.60      0.63   
RidgeClassifierCV           

# Optimzed clf

In [16]:
X_train = np.array(dataset_train["sample"].to_list())
y_train = np.array(dataset_train["label"].to_list())
X_test = np.array(dataset_test["sample"].to_list())
y_test = np.array(dataset_test["label"].to_list())

In [17]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import NuSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
import joblib  # Para guardar los modelos

# Definición de modelos y sus respectivos hiperparámetros
models = {
    "NuSVC": (NuSVC(n_jobs=-1), {
        "nu": [0.1, 0.3, 0.5, 0.7, 0.9],
        "kernel": ["linear", "rbf", "poly", "sigmoid"],
        "degree": [2, 3, 4],
        "gamma": ["scale", "auto"]
    }),
    "SVC": (SVC(n_jobs=-1), {
        "C": [0.1, 1, 10, 100],
        "kernel": ["linear", "rbf", "poly", "sigmoid"],
        "degree": [2, 3, 4],
        "gamma": ["scale", "auto"]
    }),
    "GaussianNB": (GaussianNB(n_jobs=-1), {
        "var_smoothing": np.logspace(-9, -6, 5)
    })
}

# Diccionario para guardar los mejores modelos
best_models = {}

# Optimización y evaluación
for model_name, (model, param_grid) in models.items():
    print(f"\nOptimizing {model_name}...")
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Mejores hiperparámetros
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    
    # Evaluación en conjunto de prueba
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    print(f"Classification report for {model_name}:\n{classification_report(y_test, y_pred)}")
    
    # Guardar el mejor modelo en el diccionario
    best_models[model_name] = best_model


Optimizing NuSVC...


KeyboardInterrupt: 

In [ ]:
# Guardar el mejor modelo en un archivo (ejemplo con NuSVC)
model_to_save = "NuSVC"  # Cambiar por "SVC" o "GaussianNB" si se prefiere otro
if model_to_save in best_models:
    joblib.dump(best_models[model_to_save], f"{model_to_save}_best_model.pkl")
    print(f"Modelo {model_to_save} guardado como {model_to_save}_best_model.pkl")
else:
    print(f"No se encontró el modelo {model_to_save} en el diccionario.")
